In [2]:
from src.pipeline import VSFStableDiffusion3Pipeline

In [ ]:
import torch
pipe = VSFStableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-large-turbo",
    torch_dtype=torch.bfloat16,
).to("cuda:1")

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
pipe(
    prompt="a beautiful landscape with mountains and a river in Canada during winter",
    negative_prompt="snow",
    guidance_scale=0.0,
    num_inference_steps=8,
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 1 has a total capacity of 47.53 GiB of which 9.06 MiB is free. Process 2984241 has 16.65 GiB memory in use. Including non-PyTorch memory, this process has 30.85 GiB memory in use. Of the allocated memory 30.42 GiB is allocated by PyTorch, and 118.83 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)